# Generalizing Failure Circumstances

One central question in debugging is: _Does this bug occur in other situations, too?_ In this chapter, we present a technique that is set to _generalize_ the circumstances under which a failure occurs. The DDSET algorithm takes a failure-inducing input, breaks it into individual elements. For each element, it tries to find whether it can be replaced by others in the same category, and if so, it _generalizes_ the concrete element to the very category. The result is a _pattern_ that characterizes the failure condition: "The failure occurs for all inputs of the form `(<expr> * <expr>)`.

In [ ]:
from bookutils import YouTubeVideo
# YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* You should have read the [chapter on _delta debugging_](DeltaDebugger.ipynb).

In [ ]:
import bookutils

In [ ]:
import DeltaDebugger

# Synopsis

<!-- Automatically generated. Do not edit. -->



_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
print(int_fuzzer())
```
```python
=> 76.5

```


## A Failing Program

As with previous chapters, we use `remove_html_markup()` as our ongoing example. This function is set to remove HTML markup tags (like `<em>`) from a given string `s`, returning the plain text only. We use the version from [the chapter on asssertions](Assertions.ipynb), using an assertion as postcondition checker.

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

For the most inputs, `remove_html_markup()` works just as expected:

In [ ]:
remove_html_markup("Be <em>quiet</em>, he said")

There are inputs, however, for which it fails:

In [ ]:
BAD_INPUT = '<foo>"bar</foo>'

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError(AssertionError):
    remove_html_markup(BAD_INPUT)

In [ ]:
from bookutils import quiz

In contrast to the other chapters, our aim now is not to immediately go and debug `remove_html_markup()`. Instead, we focus on another important question: 

> Under which conditions precisely does `remove_html_markup()` fail?

This question can be generalized to

> What is the set of inputs for which `remove_html_markup()` fails?

Our plan for this is to _generalize_ concrete inputs (such as `BAD_INPUTS`) into an *abstract failure-inducing inputs*. These are patterns formed from a concrete input, but in which specific _placeholders_ indicate sets of inputs that are permitted. In the abstract failure-inducing input

```html
<opening-tag>"bar<closing-tag>
```

for instance, `<opening-tag>` and `<closing-tag>` are placeholders for opening and closing HTML tags, respectively. The pattern indicates that any opening HTML tag and closing HTML tag can be present in the input, as long as the enclosed text reads `"bar`.

Given a concrete failure-inducing input, our aim is to _generalize_ it as much as possible to such an abstract failure-inducing input. The resulting pattern should then

* capture the _circumstances_ under which the program fails;
* allow for _test generation_ by instantiating the placeholders;
* help ensuring our fix is as _general as possible_.

In [ ]:
quiz("If `s = '<foo>\"bar</foo>'` (i.e., `BAD_INPUT`), "
     "what is the value of `out` such that the assertion fails?",
    [
        '`bar`',
        '`bar</foo>`',
        '`"bar</foo>`',
        '`<foo>"bar</foo>`',
    ], '9999999 // 4999999')

## Grammars

To determine abstract failure-inducing inputs, we need means to determine and characterize _sets of inputs_ – known in computer science as _languages_. To formally describe languages, the field of *formal languages* has devised a number of *language specifications* that describe a language.  *Regular expressions* represent the simplest class of these languages to denote sets of strings: The regular expression `[a-z]*`, for instance, denotes a (possibly empty) sequence of lowercase letters.  *Automata theory* connects these languages to automata that accept these inputs; *finite state machines*, for instance, can be used to specify the language of regular expressions.

Regular expressions are great for not-too-complex input formats, and the associated finite state machines have many properties that make them great for reasoning.  To specify more complex inputs, though, they quickly encounter limitations.  At the other end of the language spectrum, we have *universal grammars* that denote the language accepted by *Turing machines*.  A Turing machine can compute anything that can be computed; and with Python being Turing-complete, this means that we can also use a Python program $p$ to specify or even enumerate legal inputs. But then, computer science theory also tells us that each such program has to be written specifically for the input to be considered, which is not the level of automation we want.

The middle ground between regular expressions and Turing machines is covered by *grammars*.  Grammars are among the most popular (and best understood) formalisms to formally specify input languages.  Using a grammar, one can express a wide range of the properties of an input language.  Grammars are particularly great for expressing the *syntactical structure* of an input, and are the formalism of choice to express nested or recursive inputs.  The grammars we use are so-called *context-free grammars*, one of the easiest and most popular grammar formalisms.

A grammar is defined as a mapping of _nonterminal_ symbols (denoted in `<angle brackets>` to lists of alternative _expansions_, which are strings containing _terminal_ symbols and possibly more _nonterminal_ symbols. To make the writing of grammars as simple as possible, we adopt the [fuzzingbook](https://www.fuzzingbook.org/) format that is based on strings and lists.

In [ ]:
import fuzzingbook

Fuzzingbook grammars take the format of a _mapping_ between symbol names and expansions, where expansions are _lists_ of alternatives.  A one-rule grammar for digits thus takes the form

In [ ]:
DIGIT_GRAMMAR = {
    "<start>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

which means that the `<start>` symbol can be expanded into any of the digits listed.

A full grammar for arithmetic expressions looks like this:

In [ ]:
EXPR_GRAMMAR = {
    "<start>":
        ["<expr>"],

    "<expr>":
        ["<term> + <expr>", "<term> - <expr>", "<term>"],

    "<term>":
        ["<factor> * <term>", "<factor> / <term>", "<factor>"],

    "<factor>":
        ["+<factor>",
         "-<factor>",
         "(<expr>)",
         "<integer>.<integer>",
         "<integer>"],

    "<integer>":
        ["<digit><integer>", "<digit>"],

    "<digit>":
        ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
}

From such a grammar, one can easily generate inputs that conform to the grammar.

In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

In [ ]:
simple_expr_fuzzer = GrammarFuzzer(EXPR_GRAMMAR)

In [ ]:
for i in range(10):
    fuzz_expr = simple_expr_fuzzer.fuzz()
    print(fuzz_expr)

Nonterminals as found in the grammar make natural _placeholders_ in abstract failure-inducing inputs. If we know, for instance, that it is not just the concrete failure-inducing input

```python
(2 * 3)
```

but the abstract failure-inducing input

```html
(<expr> * <expr>)
```

that causes the failure, we immediately see that the error is due to the multiplication operator rather than its operands.

Coming back to our `remove_html_markup()` example, let us create a simple grammar for HTML expressions. A `<html>` element is either plain text or tagged text.

In [ ]:
SIMPLE_HTML_GRAMMAR = {
    "<start>":
        ["<html>"],

    "<html>":
        ["<plain-text>", "<tagged-text>"],
}

Plain text is a simple (possibly empty) sequence of letter, digits, punctuation, and whitespace. (Note how `<plain-text>` is either empty or some character followed by more plain text.) The characters `<` and `>` are not allowed, though.

In [ ]:
import string

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<plain-text>":
        ["", "<plain-char><plain-text>"],

    "<plain-char>":
        ["<letter>", "<digit>", "<other>", "<whitespace>"],

    "<letter>": list(string.ascii_letters),
    "<digit>": list(string.digits),
    "<other>": list(string.punctuation.replace('<', '').replace('>', '')),
    "<whitespace>": list(string.whitespace)
})

Tagged text is a bit more complicated. We have opening tags `<foo>`, followed by some more HTML material, and then closed by a closing tag `</foo>`. (We do not insist that the two tags match.) A self-closing tag has the form `<br/>`. For compatibility reasons, we also allow just opening tags without closing tags, as in `<img>`.

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<tagged-text>":
        ["<opening-tag><html><closing-tag>",
         "<self-closing-tag>",
         "<opening-tag>"],
})

Since the characters `<` and `>` are already reserved for denoting nonterminal symbols, we use the special nonterminal symbols `<lt>` and `<gt>` that expand into `<` and `>`, respectively,

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<opening-tag>":
        ["<lt><id><gt>",
         "<lt><id><attrs><gt>"],

    "<lt>": [ "<" ],
    "<gt>": [ ">" ],

    "<id>":
        ["<letter>", "<id><letter>", "<id><digit>"],

    "<closing-tag>":
        ["<lt>/<id><gt>"],

    "<self-closing-tag>":
        ["<lt><id><attrs>/<gt>"],
})

Finally, HTML tags can have attributes, which are enclosed in quotes.

In [ ]:
SIMPLE_HTML_GRAMMAR.update({
    "<attrs>":
        ["<attr>", "<attr><attrs>" ],

    "<attr>":
        [" <id>='<plain-text>'",
         ' <id>="<plain-text>"'],
})

Again, we can generate inputs from the grammar.

In [ ]:
simple_html_fuzzer = GrammarFuzzer(SIMPLE_HTML_GRAMMAR)

In [ ]:
for i in range(10):
    fuzz_html = simple_html_fuzzer.fuzz()
    print(repr(fuzz_html))

Such inputs, of course, are great for systematic testing. Our sister book, [the fuzzing book](https://www.fuzzingbook.org/), covers these and more.

## Derivation Trees

To produce inputs from a grammar, the fuzzingbook `GrammarFuzzer` makes use of a structure called a *derivation tree* (also known as *syntax tree*). A derivation tree encodes the individual expansion steps undertaken while producing the output.

Let us illustrate derivation trees by example, using the last HTML output we produced.

In [ ]:
fuzz_html

The `GrammarFuzzer` attribute `derivation_tree` holds the last tree used to produced this input. We can visualize the tree as follows:

In [ ]:
from fuzzingbook.GrammarFuzzer import display_tree

In [ ]:
# ignore
def display_tree(tree):
    def graph_attr(dot):
        dot.attr('node', shape='plain')
        dot.attr('node', fontname="'Fira Mono', 'Source Code Pro', 'Courier', monospace")
        
    def node_attr(dot, nid, symbol, ann):
        fuzzingbook.GrammarFuzzer.default_node_attr(dot, nid, symbol, ann)
        if symbol.startswith('<'):
            dot.node(repr(nid), fontcolor='#0060a0')
        else:
            dot.node(repr(nid), fontcolor='#00a060')
        dot.node(repr(nid), scale='2')
    
    return fuzzingbook.GrammarFuzzer.display_tree(tree,
        node_attr=node_attr,
        graph_attr=graph_attr)

In [ ]:
display_tree(simple_html_fuzzer.derivation_tree)

From top to bottom, we see that the input was constructed from a `<start>` symbol, which then expanded into `html`, which then expanded into HTML text, and so on. Multiple children in a tree stand for a concatenation of individual symbols.

Internally, these trees come as pairs `(symbol, children)`, where `symbol` is the name of a node (say, `<html>`), and `children` is a (possibly empty) list of subtrees. Here are the topmost nodes of the above tree:

In [ ]:
import pprint

In [ ]:
pp = pprint.PrettyPrinter(depth=7)
pp.pprint(simple_html_fuzzer.derivation_tree)

To produce abstract failure-inducing patterns, we will work on this very structure. The idea is to

1. systematically replace subtrees by other, generated, compatible subtrees (e.g. replace one `<html>` subtree in the concrete input by some other generated `<html>` subtree);
2. see whether these subtrees also result in failures; and
3. if they do, use the nonterminal (`<html>`) as a placeholder in the pattern.

This will involve some subtree manipulation, construction, and finally testing. First of all, though, we need to be able to turn an _existing input_ into a derivation tree.

## Parsing

The activity of creating a structure out of an unstructured input is called _parsing_. Generally speaking, a _parser_ uses a _grammar_ to create a _derivation tree_ (also called *parse tree* in parsing contexts) from a string input.

Again, there's a whole body of theory (and practice!) around constructing parsers. We make our life simple by using an existing parser (again, from [the fuzzing book](https://www.fuzzingbook.org/Parser.html)), which does just what we want. The `EarleyParser` is instantiated with a grammar such as `SIMPLE_HTML_GRAMMAR`:

In [ ]:
from fuzzingbook.Parser import EarleyParser

In [ ]:
simple_html_parser = EarleyParser(SIMPLE_HTML_GRAMMAR)

Its method `parse()` returns an iterator over multiple possible derivation trees.  (There can be multiple trees because the grammar could be ambiguous). We are only interested in the first such tree. Let us parse `BAD_INPUT` and inspect the resulting ~parse tree~ ~syntax tree~ derivation tree:

In [ ]:
bad_input_tree = list(simple_html_parser.parse(BAD_INPUT))[0]

In [ ]:
display_tree(bad_input_tree)

This derivation tree has the same structure as the one created from our `GrammarFuzzer` above. We see how the `<tagged-text>` is composed of three elements:

1. an`<opening-tag>` (`<foo>`);
2. a `<html>` element which becomes `<plain-text>` (`"bar`); and
3. a `<closing-tag>` (`</foo>`).

We can easily turn the tree back into a string. The method `tree_to_string()` traverses the tree left-to-right and joins all nonterminal symbols.

In [ ]:
from fuzzingbook.GrammarFuzzer import tree_to_string, all_terminals

In [ ]:
tree_to_string(bad_input_tree)

In [ ]:
assert tree_to_string(bad_input_tree) == BAD_INPUT

With this, we can now

* parse an input into a tree structure;
* (re-)create parts of the tree structure; and
* turn the tree back into an input string.

## Mutating the Tree

We introduce a class `TreeMutator` that is set to mutate a tree. Its constructor takes a grammar and a tree.

In [ ]:
from fuzzingbook.Grammars import is_valid_grammar

In [ ]:
class TreeMutator:
    """Grammar-based mutations of derivation trees."""
    
    def __init__(self, grammar, tree, fuzzer=None, log=False):
        """Constructor. 
        `grammar` is the underlying grammar; 
        `tree` is the tree to work on.
        `fuzzer` is the grammar fuzzer to use (default: `GrammarFuzzer`)
        """

        assert is_valid_grammar(grammar)
        self.grammar = grammar
        self.tree = tree
        self.log = log
        
        if fuzzer is None:
            fuzzer = GrammarFuzzer(grammar)
        self.fuzzer = fuzzer

### Referencing Subtrees

To reference individual elements in the tree, we introduce the concept of a _path_. A path is a list of numbers indicating the children (starting with 0) we should follow. A path `[0, 0, 0, ..., 0]` stands for the leftmost child in a tree.

The method `get_subtree()` returns the subtree for a given path.

In [ ]:
class TreeMutator(TreeMutator):
    def get_subtree(self, path, tree=None):
        """Access a subtree based on `path` (a list of children numbers)"""
        if tree is None:
            tree = self.tree

        node, children = tree  # FIXME: should be symbol

        if not path:
            return tree

        return self.get_subtree(path[1:], children[path[0]])

Here's `get_subtree()` in action. We instantiate a `TreeMutator` on the `BAD_INPUT` tree as shown above and return the element at the path `[0, 0, 1, 0]` – i.e. follow the leftmost edge twice, than the second-to-leftmost edge, then the leftmost edge again. This gives us the `<plain-text>` subtree representing the string `"bar`:

In [ ]:
def bad_input_tree_mutator():
    return TreeMutator(SIMPLE_HTML_GRAMMAR, bad_input_tree, log=2)    

In [ ]:
plain_text_subtree = bad_input_tree_mutator().get_subtree([0, 0, 1, 0])
pp.pprint(plain_text_subtree)

In [ ]:
tree_to_string(plain_text_subtree)

In [ ]:
# ignore
def primes_generator():
    # Adapted from https://www.python.org/ftp/python/doc/nluug-paper.ps
    primes = [2]
    yield 2
    i = 3
    while True:
        for p in primes:
            if i % p == 0 or p * p > i:
                break

        if i % p != 0:
            primes.append(i)
            yield i

        i += 2

In [ ]:
# ignore
prime_numbers = primes_generator()

In [ ]:
quiz("In `bad_input_tree`, what is "
     " the subtree at the path `[0, 0, 2, 1]` as string?", 
    [
        f"`{tree_to_string(bad_input_tree_mutator().get_subtree([0, 0, 2, 0]))}`",
        f"`{tree_to_string(bad_input_tree_mutator().get_subtree([0, 0, 2, 1]))}`",
        f"`{tree_to_string(bad_input_tree_mutator().get_subtree([0, 0, 2]))}`",
        f"`{tree_to_string(bad_input_tree_mutator().get_subtree([0, 0, 0]))}`",
    ], 'next(prime_numbers)', globals()
    )

### Creating new Subtrees

The method `new_tree()` creates a new subtree for the given `<start_symbol>` according to the rules of the grammar. It invokes `expand_tree()` on the given `GrammarFuzzer` – a method that takes an initial (empty) tree and expands it until no more expansions are left.

In [ ]:
class TreeMutator(TreeMutator):
    def new_tree(self, start_symbol):
        """Create a new subtree for <start_symbol>."""

        if self.log >= 2:
            print(f"Creating new tree for {start_symbol}")
            
        tree = (start_symbol, None)
        return self.fuzzer.expand_tree(tree)

Here is an example of `new_tree()`:

In [ ]:
plain_text_tree = bad_input_tree_mutator().new_tree('<plain-text>')
display_tree(plain_text_tree)

In [ ]:
tree_to_string(plain_text_tree)

### Mutating the Tree

With us now being able to 
* access a particular path in the tree (`get_subtree()`) and
* create a new subtree (`new_tree()`),

we can mutate the tree at a given path. This is the task of `mutate()`.

In [ ]:
class TreeMutator(TreeMutator):
    def mutate(self, path, tree=None):
        """Return a new tree mutated at `path`"""
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return self.new_tree(node)

        head = path[0]
        new_children = (children[:head] +
                        [self.mutate(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

Here is an example of `mutate()` in action. We mutate `bad_input_tree` at the path `[0, 0, 1, 0]` – that is, `<plain-text>`:

In [ ]:
mutated_tree = bad_input_tree_mutator().mutate([0, 0, 1, 0])
display_tree(mutated_tree)

We see that the `<plain-text>` subtree is now different, which also becomes evident in the string representation.

In [ ]:
tree_to_string(mutated_tree)

## Generalizing Trees

Now for the main part – finding out which parts of a tree can be generalized. Our idea is to _test_ a finite number of mutations to a subtree (say, 10). If all of these tests fail as well, then we assume we can generalize the subtree to a placeholder.

We introduce a class `TreeGeneralizer` for this purpose. On top of `grammar` and `tree` already used for the `TreeMutator` constructor, the `TreeGeneralizer` also takes a `test` function.

In [ ]:
class TreeGeneralizer(TreeMutator):
    """Determine which parts of a derivation tree can be generalized."""
    
    def __init__(self, grammar, tree, test,
                 max_tries_for_generalization=10,
                 **kwargs):
        """Constructor. `grammar` and `tree` are as in `TreeMutator`.
        `test` is a function taking a string that either
          * raises an exception, indicating test failure;
          * or not, indicating test success.
        `max_tries_for_generalization` is the number of times
          an instantiation has to fail before it is generalized."""
        
        super().__init__(grammar, tree, **kwargs)
        self.test = test
        self.max_tries_for_generalization = max_tries_for_generalization

The `test` function is used in `test_tree()`, returning `False` if the test fails (raising an exception), and `True` if the test passes (no exception).

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def test_tree(self, tree):
        """Return True if testing `tree` passes, else False"""
        s = tree_to_string(tree)
        if self.log:
            print(f"Testing {repr(s)}...", end="")
        try:
            self.test(s)
        except Exception as exc:
            if self.log:
                print(f"FAIL ({type(exc).__name__})")
            ret = False
        else:
            if self.log:
                print(f"PASS")
            ret = True

        return ret

### Testing for Generalization

The `can_generalize()` method brings the above methods together. It creates a number of tree mutations at the given path, and returns True if all of them produce a failure. (Note that this is not as sophisticated as our [delta debugger](DeltaDebugger.ipynb) implementation, which also checks that the _same_ error occurs.)

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def can_generalize(self, path, tree=None):
        """Return True if the subtree at `path` can be generalized."""
        for i in range(self.max_tries_for_generalization):
            mutated_tree = self.mutate(path, tree)
            if self.test_tree(mutated_tree):
                # Failure no longer occurs; cannot abstract
                return False
            
        return True

Let us put `TreeGeneralizer` into action. We can directly use `remove_html_markup()` as test function.

In [ ]:
def bad_input_tree_generalizer(**kwargs):
    return TreeGeneralizer(SIMPLE_HTML_GRAMMAR, bad_input_tree,
        remove_html_markup, **kwargs)    

On our `BAD_INPUT` (and its tree), can we generalize the root `<html>` node? In other words, does the failure occur for all possible `<html>` inputs?

In [ ]:
bad_input_tree_generalizer(log=True).can_generalize([0])

The answer is no. The first alternative passes the test; hence no generalization.

How about the middle `<plain_text>` part? Can we generalize this?

In [ ]:
bad_input_tree_generalizer(log=True).can_generalize([0, 0, 1, 0])

The answer is no – just as above.

How about the closing tag? Can we generalize this one?

In [ ]:
bad_input_tree_generalizer(log=True).can_generalize([0, 0, 2])

Yes, we can! All alternate instantiations of `<closing-tag>` result in a failure.

In [ ]:
quiz("Is this also true for `<opening-tag>`?",
     [
         "Yes",
         "No"
     ], '("No" == "No") + ("No" is "No")')

Note that the above does not hold for `<opening-tag>`. If the attribute value contains a quote character, it may extend to the end of the input. 

In [ ]:
bad_input_tree_generalizer().can_generalize([0, 0, 0])

This, however, becomes apparent only after a larger number of tests.

In [ ]:
bad_input_tree_generalizer(max_tries_for_generalization=100).can_generalize([0, 0, 0])

We see that our approach may _overgeneralize_ – producing a generalization that may be too lenient. In practice, this is not too much of a problem, as we would be interested in characterizing cases that trigger the failure, rather than characterizing a small subset that does not trigger the failure.

### Generalizable Paths

Using `can_generalize()`, we can devise a method `generalizable_paths()` that returns all paths in the tree that can be generalized.

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def find_paths(self, predicate, path=None, tree=None):
        """Return a list of all paths for which `predicate` holds.
        `predicate` is a function `predicate`(`path`, `tree`), where
         `path` denotes a subtree in `tree`. If `predicate()` returns
         True, `path` is included in the returned list."""

        if path is None:
            path = []
        if tree is None:
            tree = self.tree
            
        node, children = self.get_subtree(path)

        if predicate(path, tree):
            if self.log:
                node, children = self.get_subtree(path)
            return [path]

        paths = []
        for i, child in enumerate(children):
            child_node, _ = child
            if child_node in self.grammar:
                paths += self.find_paths(predicate, path + [i])

        return paths        
    
    def generalizable_paths(self):
        """Return a list of all paths whose subtrees can be generalized."""
        return self.find_paths(self.can_generalize)

Here is `generalizable_paths()` in action. We obtain all (paths to) subtrees that can be generalized:

In [ ]:
bad_input_generalizable_paths = \
    bad_input_tree_generalizer().generalizable_paths()
bad_input_generalizable_paths

To convert these subtrees into abstract failure-inducing patterns, the method `generalize_path()` returns a copy of the tree with the subtree replaced by a nonterminal without children:

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def generalize_path(self, path, tree=None):
        """Return a copy of the tree in which the subtree at `path`
        is generalized (= replaced by a nonterminal without children)"""
        if tree is None:
            tree = self.tree

        symbol, children = tree

        if not path:
            return symbol, None  # Nonterminal without children

        head = path[0]
        new_children = (children[:head] +
                        [self.generalize_path(path[1:], children[head])] +
                        children[head + 1:])
        return symbol, new_children

The function `all_terminals()` expands these placeholders:

In [ ]:
all_terminals(bad_input_tree_generalizer().generalize_path([0, 0, 0]))

Finally, the method `generalize()` obtains a tree in which all generalizable paths actually are generalized:

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def generalize(self):
        """Returns a copy of the tree in which all generalizable subtrees
        are generalized (= replaced by nonterminals without children)"""
        tree = self.tree
        for path in self.generalizable_paths():
            tree = self.generalize_path(path, tree)
            
        return tree

In [ ]:
abstract_failure_inducing_input = bad_input_tree_generalizer().generalize()

This gives us the final generalization of `BAD_INPUT`. In the abstract failure-inducing input, all generalizable elements are generalized.

In [ ]:
all_terminals(abstract_failure_inducing_input)

We see that to obtain the failure, it suffices to have an `<opening-tag>`, followed by a quote and (any) `<plain-text>` and (any) `<closing-tag>`. Clearly, all that it takes to produce the failure is to have a double quote in the plain text.

Also note how this diagnosis was reached through _experiments_ only – just as with [delta debugging](DeltaDebugger.ipynb), we could treat the program under test as a black box. In contrast to delta debugging, however, we obtain an _abstraction_ that generalizes the circumstances under which a given failure occurs.

## Fuzzing with Patterns

One neat side effect of abstract failure-inducing patterns is that they can be easily instantiated into further test cases, all set to reproduce the failure in question. This gives us a test suite we can later test our fix against.

The method `fuzz_tree()` takes a tree representing an abstract failure-inducing input and instantiates all missing subtrees.

In [ ]:
import copy

In [ ]:
class TreeGeneralizer(TreeGeneralizer):
    def fuzz_tree(self, tree):
        """Return an instantiated copy of `tree`."""
        tree = copy.deepcopy(tree)
        return self.fuzzer.expand_tree(tree)

In [ ]:
bitg = bad_input_tree_generalizer()
for i in range(10):
    print(all_terminals(bitg.fuzz_tree(abstract_failure_inducing_input)))

We can take these inputs and see whether they reproduce the failure in question:

In [ ]:
successes = 0
failures = 0
trials = 1000

for i in range(trials):
    test_input = all_terminals(
        bitg.fuzz_tree(abstract_failure_inducing_input))
    try:
        remove_html_markup(test_input)
    except AssertionError:
        successes += 1
    else:
        failures += 1

In [ ]:
successes, failures

We get an overall failure rate of ~99%, which is not bad at all.

In [ ]:
failures / 1000

In our case, it is _overgeneralization_ (as discussed above) that is responsible for not reaching a 100% rate. (In all generality, we're trying to approximate the behavior of a Turing machine with a context free grammar, which is, well, always an approximation.) However, even a lower rate would still be useful, as any additional test case that reproduces a failure helps in ensuring the final fix is complete.

## Putting it all Together

Let us now put together all this in a more convenient package that does not require the user to parse and unparse derivation trees.

Our `DDSetDebugger` is modeled after the `DeltaDebugger` from [the chapter on delta debugging](DeltaDebugger.ipynb). It is to be used as

```python
with DDSetDebugger(grammar) as dd:
    some_failing_function(...)
```

After that, evaluating `dd` yields a generalized abstract failure-inducing input as a string.

Since `DDSetDebugger` accepts only one grammar, the function to be debugged should have exactly one string argument (besides other arguments); this string must fit the grammar.

### Constructor

The constructor puts together the various components. It allows for customization by subclassing.

In [ ]:
from DeltaDebugger import CallCollector, is_reducible

In [ ]:
import copy

In [ ]:
class DDSetDebugger(CallCollector):
    """Debugger implementing the DDSET algorithm
    for abstracting failure-inducing inputs"""
    
    def __init__(self, grammar, 
                 generalizer_class=TreeGeneralizer,
                 parser=None,
                 **kwargs):
        """Constructor.
        `grammar` is an input grammar in fuzzingbook format.
        `generalizer_class` is the tree generalizer class to use
          (default: TreeGeneralizer)
        `parser` is the parser to use (default: `EarleyParser(grammar)`).

        All other args are passed to the tree generalizer, notably
        `fuzzer` is the fuzzer to use (default: `GrammarFuzzer`)
        `log` enables debugging output if set to True.
        """
        super().__init__()
        self.grammar = grammar
        assert is_valid_grammar(grammar)

        self.generalizer_class = generalizer_class

        if parser is None:
            parser = EarleyParser(grammar)
        self.parser = parser
        self.kwargs = kwargs
        
        self.generalized_trees = None
        self.generalized_args = None

### Generalizing Arguments

The method `generalize()` is the many entry point. For all string arguments collected in the first function call, it generalizes the arguments and returns an abstract failure-inducing string.

In [ ]:
class DDSetDebugger(DDSetDebugger):
    def generalize(self):
        """Generalize arguments seen. For each function argument,
        produce an abstract failure-inducing input that characterizes
        the set of inputs for which the function fails."""
        self.generalized_args = copy.deepcopy(self.args())
        self.generalized_trees = {}

        for arg in self.args():
            def test(value):
                return self.call({arg: value})

            value = self.args()[arg]
            if isinstance(value, str):
                tree = list(self.parser.parse(value))[0]
                gen = self.generalizer_class(self.grammar, tree, test, 
                                             **self.kwargs)
                generalized_tree = gen.generalize()
                self.generalized_trees[arg] = generalized_tree
                self.generalized_args[arg] = all_terminals(generalized_tree)

        return self.generalized_args

In [ ]:
class DDSetDebugger(DDSetDebugger):
    def __repr__(self):
        """Return a string representation of the minimized call."""
        return self.format_call(self.generalize())

Here is an example of how `DDSetDebugger` would be used on our `BAD_INPUT` example:

In [ ]:
with DDSetDebugger(SIMPLE_HTML_GRAMMAR) as dd:
    remove_html_markup(BAD_INPUT)
dd

### Fuzzing

The `fuzz()` method produces more instantiations.

In [ ]:
class DDSetDebugger(DDSetDebugger):
    def fuzz_args(self):
        if self.generalized_trees is None:
            self.generalize()
            
        args = copy.deepcopy(self.generalized_args)
        for arg in args:
            def test(value):
                return self.call({arg: value})

            if arg not in self.generalized_trees:
                continue

            tree = self.generalized_trees[arg]
            gen = self.generalizer_class(self.grammar, tree, test, 
                                             **self.kwargs)
            instantiated_tree = gen.fuzz_tree(tree)
            args[arg] = all_terminals(instantiated_tree)
            
        return args
    
    def fuzz(self):
        return self.format_call(self.fuzz_args())

In [ ]:
with DDSetDebugger(SIMPLE_HTML_GRAMMAR) as dd:
    remove_html_markup(BAD_INPUT)

In [ ]:
dd.fuzz()

In [ ]:
dd.fuzz()

In [ ]:
dd.fuzz()

## More Examples

### Square Root

In [ ]:
from Assertions import square_root

In [ ]:
INT_GRAMMAR = {
    "<start>":
        ["<int>"],

    "<int>":
        ["<positive-int>", "-<positive-int>"],

    "<positive-int>":
        ["<digit>", "<nonzero-digit><positive-int>"],

    "<nonzero-digit>": list("123456789"),
    
    "<digit>": list(string.digits),
}

In [ ]:
def square_root_test(s):
    return square_root(int(s))

In [ ]:
with DDSetDebugger(INT_GRAMMAR, log=True) as dd:
    square_root_test("-1")

In [ ]:
dd.generalized_args()['s']

### Middle

In [ ]:
from StatisticalDebugger import middle

In [ ]:
XYZ_GRAMMAR = {
    "<start>":
        ["<int>, <int>, <int>"],

    "<int>":
        ["<positive-int>", "-<positive-int>"],

    "<positive-int>":
        ["<digit>", "<nonzero-digit><positive-int>"],

    "<nonzero-digit>": list("123456789"),
    
    "<digit>": list(string.digits),
}

In [ ]:
def middle_test(s):
    x, y, z = eval(s)
    assert middle(x, y, z) == sorted([x, y, z])[1]

In [ ]:
with ExpectError(AssertionError):
    middle_test("2, 1, 3")

In [ ]:
with DDSetDebugger(XYZ_GRAMMAR, log=True) as dd:
    middle_test("2, 1, 3")

In [ ]:
dd.generalized_args()['s']

## Synopsis

_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

https://rahul.gopinath.org/post/2020/07/15/ddset/

https://rahul.gopinath.org/post/2020/08/03/simple-ddset/



_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

### Exercise 1: Generalization and Specialization

Consider the abstract failure-inducing input for `BAD_INPUT` we determined:

In [ ]:
all_terminals(abstract_failure_inducing_input)

1. How does it change if you increase the number of test runs, using `max_tries_for_generalization`?
2. What is the success rate of the new pattern?

**Solution.** We can compute this by increasing `max_tries_for_generalization`:

In [ ]:
more_precise_bitg = \
    bad_input_tree_generalizer(max_tries_for_generalization=100)

more_precise_abstract_failure_inducing_input = \
    more_precise_bitg.generalize()

In [ ]:
all_terminals(more_precise_abstract_failure_inducing_input)

We see that we still have an opening tag; however, it no longer assumes attributes.

The success rate can be computed as before:

In [ ]:
successes = 0
failures = 0
trials = 1000

for i in range(trials):
    test_input = all_terminals(
        more_precise_bitg.fuzz_tree(
            more_precise_abstract_failure_inducing_input))
    try:
        remove_html_markup(test_input)
    except AssertionError:
        successes += 1
    else:
        failures += 1

In [ ]:
successes, failures

In [ ]:
failures / 1000

We see that the failure rate is now less than 99%, which is better than before. On the other hand, the pattern is now overly _special_, since there are `<opening-tags>` with attributes such that the failure occurs (but also some that cancel out the error).

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_

## Reducing Trees

In [ ]:
from DeltaDebugger import DeltaDebugger

In [ ]:
import copy

In [ ]:
from IPython.display import display

In [ ]:
class TreeHDDReducer(TreeGeneralizer):
    def _reduce(self, path, tree):
        """This is HDD"""

        node, children = self.get_subtree(path, tree)
            
        if len(path) >= 1:
            parent, parent_children = self.get_subtree(path[:-1], tree)
 
            assert parent_children[path[-1]] == (node, children)

            def test_children(children):
                parent_children[path[-1]] = (node, children)
                s = tree_to_string(tree)
                self.test(s)

            with DeltaDebugger() as dd:
                test_children(children)
            
            # display(display_tree(tree))

            children = dd.min_args()['children']
            parent_children[path[-1]] = (node, children)
        
        for i, child in enumerate(children):
            self._reduce(path + [i], tree)
            
        return tree

    def reduce(self):
        return self._reduce([], self.tree)

In [ ]:
def bad_input_tree_hdd_reducer():
    return TreeHDDReducer(SIMPLE_HTML_GRAMMAR, copy.deepcopy(bad_input_tree),
                       remove_html_markup, log=True)    

In [ ]:
all_terminals(bad_input_tree_hdd_reducer().reduce())

## More Reducing Trees

In [ ]:
class TreeReducer(TreeGeneralizer):
    def new_min_tree(self, start_symbol):
        if self.log >= 2:
            print(f"Creating new minimal tree for {start_symbol}")

        fuzzer = GrammarFuzzer(self.grammar, start_symbol=start_symbol,
                               min_nonterminals=0,
                               max_nonterminals=0)
        fuzzer.fuzz()
        return fuzzer.derivation_tree

In [ ]:
def bad_input_tree_reducer():
    return TreeReducer(SIMPLE_HTML_GRAMMAR, bad_input_tree,
                       remove_html_markup, log=2)    

In [ ]:
tree_to_string(bad_input_tree_reducer().new_min_tree('<start>'))

In [ ]:
class TreeReducer(TreeReducer):
    def reduce_path(self, path, tree=None):
        if tree is None:
            tree = self.tree

        node, children = tree

        if not path:
            return self.new_min_tree(node)

        head = path[0]
        new_children = (children[:head] +
                        [self.reduce_path(path[1:], children[head])] +
                        children[head + 1:])
        return node, new_children

In [ ]:
tree_to_string(bad_input_tree_reducer().reduce_path([0, 0, 1, 0]))

In [ ]:
class TreeReducer(TreeReducer):
    def can_reduce(self, path, tree=None):
        reduced_tree = self.reduce_path(path, tree)
        if self.test_tree(reduced_tree):
            # Failure no longer occurs; cannot reduce
            return False

        return True

In [ ]:
class TreeReducer(TreeReducer):
    def reducible_paths(self):
        return self.find_paths(self.can_reduce)

In [ ]:
bad_input_tree_reducer().reducible_paths()

In [ ]:
class TreeReducer(TreeReducer):
    def reduce(self):
        tree = self.tree
        for path in self.reducible_paths():
            tree = self.reduce_path(path, tree)
            
        return tree

In [ ]:
all_terminals(bad_input_tree_reducer().reduce())